In [109]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [110]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [111]:
path = untar_data(URLs.MNIST)

In [112]:
#hide
Path.BASE_PATH = path

In [113]:
def create_mask(index: int, length: int):
    index = torch.tensor([index])
    src = torch.tensor([1])
    z = torch.zeros(length, dtype = src.dtype)
    return z.scatter_(0, index, src)

In [129]:
train_x = torch.cat(
    [torch.stack([tensor(Image.open(o)) for o in (path/'training'/f'{num}').ls()]).float()/255.0 for num in range(0, 10)]
).view(-1, 28*28)

train_y = torch.cat(
    [create_mask(i, 10).repeat(len((path/'training'/f'{i}').ls()), 1) for i in range(0, 10)] 
)

train_x.shape, train_y.shape

(torch.Size([60000, 784]), torch.Size([60000, 10]))

In [130]:
valid_x = torch.cat(
    [torch.stack([tensor(Image.open(o)) for o in (path/'testing'/f'{num}').ls()]).float()/255.0 for num in range(0, 10)]
).view(-1, 28*28)

valid_y = torch.cat(
    [create_mask(i, 10).repeat(len((path/'testing'/f'{i}').ls()), 1) for i in range(0, 10)] 
)

valid_x.shape, valid_y.shape

(torch.Size([10000, 784]), torch.Size([10000, 10]))

In [131]:
# valid_3_tens = torch.stack([tensor(Image.open(o)) 
#                             for o in (path/'valid'/'3').ls()])
# valid_3_tens = valid_3_tens.float()/255
# valid_7_tens = torch.stack([tensor(Image.open(o)) 
#                             for o in (path/'valid'/'7').ls()])
# valid_7_tens = valid_7_tens.float()/255


# valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
# valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))
valid_dl = DataLoader(valid_dset, batch_size=256)

# train_3_tens = torch.stack([tensor(Image.open(o)) for o in (path/'train'/'3').ls()]).float()/255.0
# train_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/'train'/'7').ls()]).float()/255.0

# train_x = torch.cat([train_3_tens, train_7_tens]).view(-1, 28*28)
# train_y = tensor([1]*len(train_3_tens) + [0]*len(train_7_tens)).unsqueeze(1)
train_dset = list(zip(train_x,train_y))
train_dl = DataLoader(train_dset, batch_size=256)

dls = DataLoaders(train_dl, valid_dl)

In [132]:
x, y = dls.one_batch()
x.shape, y.shape

(torch.Size([256, 784]), torch.Size([256, 10]))

> TODO: HERE

In [32]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

In [33]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

In [86]:
class MyLearner:
    def __init__(self, dataloaders, model, opt_func, loss_func, metrics):
        self.dataloaders = dataloaders.cuda()
        self.model = model.cuda()
        self.opt_func = opt_func
        self.loss_func = loss_func
        self.metric = metrics
        # print(self.model.device())
        # print(self.dataloaders.device)

    def fit(self, num_epochs, learning_rate):
        opt = self.opt_func(self.model.parameters(), learning_rate)
        print(opt)
        for i in range(num_epochs):
            self.train_epoch(opt)
            print(self.validate_epoch(), end=' ')
        return None

    def train_epoch(self, opt):
        train_dl = self.dataloaders[0]
        for xb,yb in train_dl:
            self.calc_grad(xb, yb)
            opt.step()
            opt.zero_grad()

    def validate_epoch(self):
        valid_dl = self.dataloaders[1]
        met = [self.metric(self.model(xb), yb) for xb,yb in valid_dl]
        return round(torch.stack(met).mean().item(), 4)

    def calc_grad(self, data_batch, label_batch):
        # print(data_batch.shape)
        pred_batch = self.model(data_batch)
        loss = self.loss_func(pred_batch, label_batch)
        # print(loss)
        loss.backward()

In [87]:
learn = MyLearner(dls, nn.Linear(28*28,1), opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [88]:
learn.fit(20, 0.1)

0.5562 0.8765 0.937 0.9575 0.9634 0.9643 0.9658 0.9678 0.9678 0.9687 0.9697 0.9702 0.9707 0.9707 0.9717 0.9717 0.9717 0.9717 0.9717 0.9726 